### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [2]:
datatracks = pd.read_csv('../data/clean/api_tracks1_clean.csv', encoding='latin1')

In [3]:
datatracks.head(2)

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [6]:
track_ids = datatracks['track_id'].unique().tolist()

In [7]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

### Spotify Connection 

In [8]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [9]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [10]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Tracks Information 

In [48]:
track_ids = datatracks['track_id'].unique().tolist()

In [11]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

In [17]:
#endpoint
#https://api.spotify.com/v1/audio-features/{id}

In [22]:
#test 
url = 'https://api.spotify.com/v1/audio-features/0QLyr8xasb6tgGkdPWYw8d'

In [30]:
res = rq.get(url, headers = headers)

In [31]:
resjson = res.json()
type(resjson)

dict

In [32]:
resjson.keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [29]:
resjson['danceability']

0.42

In [ ]:
# Construct a list of URLs for querying track audio features
track_feature_urls = [f'https://api.spotify.com/v1/audio-features/{track_id}' for track_id in track_ids]

# Create a list of dictionaries with track information and corresponding feature URLs
track_feature_url_list = [{'track_id': track_id, 'url': url} for track_id, url in zip(track_ids, track_feature_urls)]

In [33]:
# List to store the retrieved track data
track_data_list = []

# Function to search and retrieve track information
def search_track_info(track_url):
    time.sleep(0.30)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(track_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        # Extract necessary data from the response
        track_data_list.append({
            'track_id': data['id'],
            'track_name': data['name'],
            'album_id': data['album']['id'],
            'album_name': data['album']['name'],
            'artist_id': data['artists'][0]['id'],
            'artist_name': data['artists'][0]['name'],
            'danceability': data.get('danceability'),
            'popularity': data.get('popularity'),
            'energy': data.get('energy'),
            'acousticness': data.get('acousticness'),
            'valence': data.get('valence'),
            'instrumentalness': data.get('instrumentalness'),
            'tempo': data.get('tempo'),
            'loudness': data.get('loudness')
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        track_data_list.append({
            'track_id': None,
            'track_name': None,
            'album_id': None,
            'album_name': None,
            'artist_id': None,
            'artist_name': None,
            'danceability': None,
            'popularity': None,
            'energy': None,
            'acousticness': None,
            'valence': None,
            'instrumentalness': None,
            'tempo': None,
            'loudness': None
        })

# Execute the function for each track URL
for track_url in track_url_list:
    search_track_info(track_url)

# Create a DataFrame from the list of dictionaries
data_tracks2 = pd.DataFrame(track_data_list)

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1vuCDdqdaKc0rOTUjK9XDX
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0PswhfztcdmfYDqfoGMqc4
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3Ki7LceZVmcQINetqAbTWD
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/46u3ZbLTvEa9T6pZSEAngr
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/7nnl7d1EWOPHOlYDYYpB3M
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/060hIjL5vg3Zb0Im8zan5I
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/63sCRDH2JWuR7ZzWv8KQhS
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/08l91Hm1QMhcBf6JN3k8HH
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4ZIHoyUXYbPCDfVedr5rWh
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6HPL4ntDOnsdc1jtnX8XMJ
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/55i4volbrLIbUP1KjpRQmH
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4NIIxQd4TYjQncltQ4JGAH
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6WJS98cmvNEzTjQRD3dUTL
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/7xbO38eg1icRKIfu26Mvsz
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4Zp35RIATJCwHAKEhtiSb0
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5guVvrqQlZVTnHky6gxVOP
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6OQeoFuY8CnKnBR2iJmk2y
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2BgDUp12ytwJmLCab4VXsS
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4Rbniz5regXQjeqJjbcyNv
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6ysK7OSU1cOK6JuJyFhMgJ
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4N4nmc1gHqcOE9EdoV5vwE
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/7ERZgMLgpVhrp2D6JYy02t
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3gBE71E0cl8xRZLj8IJIuP
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6SPdaCpTYFe8ay3kdBHVD7
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/63zEvsxirbjiqCsR2MimgK
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1nFlSCBuiJkxcAC9Y0tW7w
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4P4RqGMOoHh3YF9WPMX91J
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/466u11we9prRQi1Y6O1soK
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5JwAHq9DO1pvn7ygcjuWwE
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4qPzEeJx6inDpSnzXf6QQZ
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4hFixo5KoIhq3gd1Xz471N
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5tDtfXgKd7dKzPqp8Lbgce
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5Qv2Nby1xTr9pQyjkrc94J
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/15irEKZ9D6FQqLoZ1qJ1Cx
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/02ppMPbg1OtEdHgoPqoqju
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/48X4D1FYOShPz2VF3YdfCF
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0YJ9FWWHn9EfnN0lHwbzvV
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4T1iiabe7G0UjWQJCY6NE2
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3pexR2j9j8OJdlxZ9E4PV0
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/7C8aMDbTGnxVMOmHPtfj5W
An error occurred: 429 Client Er

...

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6dPmWrkCwm4qHoS8Re5RqA
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6XaICZ1ml9SlR3aEVvaEtx
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2baCvEZheie7jcUAIK6obi
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2zmK3XAKsxdwmKbIBnLIxm
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5rJ2KxGS9UnKFuwodU5Vt9
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/53MLlxLRavqZnoOfzzaVI0
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0ueSbTguNmNRXgYToRzLSN
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3ZJekt62B2Omv2pK4gfP4a
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5VwSCuz9aJq4gZrXCHdHoO
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2b8GjnBA7mQdQDebBijIsX
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3zBQU8D4QSPJBeqkSTBLEd
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6n1Uf2ahEChZfSVo5SRPCy
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6NP30n7t2Hz8bNEWcx4Pgf
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1U94BKdM30t7KeCzT0pRWq
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1kRykRo5X0vsT9tVQ86BZB
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3CCBwpqjdar1PG6HHulmD0
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4siM2Ur4XBm3E8fdN5WCXK
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/5Nf7BaUIK8AsFTjrmT80zV
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4ZfsWvdzm5GtN7iV2cwoUa
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3iR22vE1kjbqbjiHbz10Eu
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4TdWnEyyq9YMuNXAyXKQ9V
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1qASBtwhCOyfn6f1ZY0k2Y
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/7wYOt6T5tmGKybJHaIJ0ZN
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3e21cX0CVwzkQXiHz7WUQZ
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3m2yzQqohT7BU4bLrcNAPH
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1Le5VAN5J7kWSqG5XsbWcm
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2Kb81eswqm1pdqT7wA3YDQ
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2bD74b14H9y5DRKhjim7or
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3bXiWZWR3CoZbhcsqfsMNa
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4jwSJxsXbuvOFemKOmXQov
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/139k1PT1l5oaafx12eTIT0
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3TogYi3mF7a5uLcfoY2XVZ
An error occurred: 429 Client Er

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2JpADrTwpUba4Rz9DsgtfV
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3kXbqLQOXYiTTo9CtmkqeT
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4Px4lKfCqJrZmuGy8FzGaf
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4ZEXle1wW3DZ0onFdXTddl
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4j9hcQZCuzxkN01pgjXQtm
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0i1q3pY9GGlozPo3fFyfzf
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0trERWKR1miqQ4pBixZ11C
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/6qfjc5QruQYkN3NlWUIpXg
An error occurred: 429 Client Er

In [74]:
data_tracks2.head()

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
2,1YFlTlQHKR62Sh39ctI7jX,Lucy in the Sky with Diamonds,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
3,1aBpFQAum7vESFuNnWeZZI,Getting Better,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
4,2OePklaMhBp1kCbbTq5UVk,Fixing a Hole,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [76]:
data_tracks.shape

(4818, 5)

In [77]:
data_tracks.to_csv('../data/clean/api_tracks1_clean.csv', index=False)